In [80]:
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from estimagic_stargazer.stargazer import Stargazer 
from IPython.core.display import HTML, Latex
from tabulate import tabulate
from collections import namedtuple
import numpy as np
diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data)
df.columns = ['Age', 'Sex', 'BMI', 'ABP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
df['target'] = diabetes.target

est = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:4]])).fit()
est2 = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:6]])).fit()



C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [81]:
def extract_params_from_sm(model):
    to_concat = []
    params_list = ["params", "pvalues", "bse"]
    for col in params_list:
        to_concat.append(getattr(model, col))
    to_concat.append(model.conf_int()[0])
    to_concat.append(model.conf_int()[1])
    params_df = pd.concat(to_concat, axis=1)
    params_df.columns = ["value", "pvalue", "standard_error", "ci_lower", "ci_upper"]
    return params_df


def extract_info_from_sm(model):
    info = {}
    key_values = [
        "rsquared",
        "rsquared_adj",
        "scale",
        "fvalue",
        "f_pvalue",
        "df_model",
        "df_resid",
    ]
    for kv in key_values:
        info[kv] = getattr(model, kv)
    info["dependent_variable"] = model.model.endog_names

    return info


In [82]:
# use the functions defined above to create estimation result that is namedtuple
namedtuplee = namedtuple("namedtuplee", "params info")
est3 = namedtuplee(
params=extract_params_from_sm(est2),
info={**extract_info_from_sm(est2)},
)


In [4]:
# using est3, create dictionary
est4 = {}
est4['params'] = est3.params
est4['info'] = est3.info


In [5]:
# using est3, create dictionary
est5 = {}
est5['params'] = est3.params
est5['info'] = est3.info


In [6]:
# check that code runs for non-list inputs
stargazer2=Stargazer(est3)

In [7]:
HTML(stargazer2.generate_observations_html())

Observations,442.0


In [8]:
# check stargazer works for sns regression result, dictionary and namedtuple
stargazer=Stargazer([est,est2,est4,est5])

In [9]:
HTML(stargazer.generate_observations_html())

Observations,442.0,442.0,442.0,442.0


In [17]:
mdict = {"a":5}
mdict.get("b",np.NaN)

nan

In [28]:
np.sqrt(np.nan)

nan

In [9]:
HTML(stargazer.render_html())


In [10]:
type(Latex(stargazer.render_latex()))

IPython.core.display.Latex

In [11]:
stargazer.custom_columns(['Model 1','Model 2','Model3'],[1,1,1])

In [12]:
HTML(stargazer.render_html())

In [13]:
stargazer.custom_columns('Test model name')

In [14]:
HTML(stargazer.render_html())

In [15]:

stargazer.show_model_numbers(False)

In [16]:
HTML(stargazer.render_html())

In [17]:
stargazer.significant_digits(2)

In [18]:
HTML(stargazer.render_html())

In [19]:
stargazer.show_confidence_intervals(True)

In [20]:
HTML(stargazer.render_html())

In [21]:
stargazer.covariate_order(['BMI', 'Age', 'S1', 'Sex'])

In [22]:
HTML(stargazer.render_html())

In [23]:
stargazer.rename_covariates({'Age': 'Oldness'})

In [24]:
HTML(stargazer.render_html())

In [25]:
stargazer.show_degrees_of_freedom(False)

In [26]:
HTML(stargazer.render_html())

In [27]:
stargazer.add_custom_notes(['First note', 'Second note'])

In [28]:
HTML(stargazer.render_html())

In [29]:
stargazer.significance_levels([0.1, 0.05, 0.07])

In [30]:
HTML(stargazer.render_html())

In [44]:
stargazer.append_notes(False)

In [45]:
HTML(stargazer.render_html())

# More flexible significance levels

In [64]:
res_sig=""
sig_char = "*"
listt =[0.1,0.025,0.01,0.05]
sig_levels = sorted(listt)
p_value = 0
if p_value<sig_levels[0]:
    res_sig += sig_char*len(listt)
else:
    for i in np.arange(len(sig_levels[1:]))+1:
        if sig_levels[i-1]<=p_value<sig_levels[i]:
            res_sig+=sig_char*(len(sig_levels)-i)

res_sig

'****'

## make notes compatible

In [123]:
notes_text = """
 <td colspan="{}" style="text-align: right">   """.format(3)
for i in range(len(sig_levels)-1):
    notes_text+="<sup>"+"*"*(len(sig_levels)-i)+"""</sup>p&lt;{}; """.format(sig_levels[i])
notes_text+="""<sup>*</sup>p&lt;{} </td>""".format(sig_levels[-1])
    

#### Check output

In [124]:
HTML(notes_text)

In [88]:
est3.params["sig_icon"] = pd.cut(
            est3.params["pvalue"],
            bins=[-1] + sorted(sig_levels) + [2],
            labels=["*" * (len(sig_levels)-i) for i in range(len(sig_levels) + 1)],
        )

In [111]:
%timeit est3.params["sig_icon_1"] =['*'*i for i in (len(sig_levels)-np.digitize(est3.params.pvalue,[-1] + sorted(sig_levels) + [2])+1)]

351 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [112]:
%timeit est3.params["sig_icon_2"] =pd.cut(est3.params["pvalue"], bins=[-1] + sorted(sig_levels) + [2],labels=["*" * (len(sig_levels)-i) for i in range(len(sig_levels) + 1)],)

2.32 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [117]:
np.digitize(0.03,[-2]+sig_levels+[1])

array(3, dtype=int64)

In [114]:
sig_levels

[0.01, 0.025, 0.05, 0.1]

In [118]:
[-2]+sig_levels+[1]

[-2, 0.01, 0.025, 0.05, 0.1, 1]